# platform

In [ ]:
open rust.rust_operators

In [ ]:
//// test

open testing

## fsharp

### os_platform

In [ ]:
nominal os_platform' = $'System.Runtime.InteropServices.OSPlatform'

union os_platform =
    | FreeBSD
    | Linux
    | OSX
    | Windows

inl os_platform = function
    | FreeBSD => $'`os_platform'.FreeBSD' : os_platform'
    | Linux => $'`os_platform'.Linux' : os_platform'
    | OSX => $'`os_platform'.OSX' : os_platform'
    | Windows => $'`os_platform'.Windows' : os_platform'

### run_platform

In [ ]:
inl run_platform forall t. (fn : os_platform -> (() -> t)) : t =
    inl result = dyn true
    $'let mutable _run_platform_!result : `t option = None '
    $'\n#if _FREEBSD'
    fn FreeBSD () |> emit_unit
    $'#endif\n#if _LINUX'
    fn Linux () |> emit_unit
    $'#endif\n#if _OSX'
    fn OSX () |> emit_unit
    $'#endif\n#if _WINDOWS'
    fn Windows () |> emit_unit
    $'#endif'
    $'|> fun x -> _run_platform_!result <- Some x'
    $'match _run_platform_!result with Some x -> x | None -> failwith "runtime.run_platform / _run_platform_!result=None"'

### is_os_platform

In [ ]:
inl is_os_platform (x : os_platform') : bool =
    x |> $'System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform'

### is_windows'

In [ ]:
inl is_windows' () : bool =
    run_platform function
        | Windows => fun () => true
        | _ => fun () => false

## platform

### is_windows

In [ ]:
inl is_windows () : bool =
    run_target function
        | Rust _ => fun () =>
            !\($'"cfg\!(windows)"')
        | Fsharp _ => fun () =>
            Windows |> os_platform |> is_os_platform
        | target => fun () => failwith $'$"platform.is_windows / target: {!target}"'

### get_executable_suffix

In [ ]:
inl get_executable_suffix () =
    if is_windows ()
    then ".exe"
    else ""

In [ ]:
//// test

get_executable_suffix ()

.exe

## main

In [ ]:
inl main () =
    $'let is_windows () = !is_windows ()' : ()
    $'let get_executable_suffix () = !get_executable_suffix ()' : ()